In [1]:
import numpy as np
import pandas as pd
import collections

## 1. Cargamos los .json a DataFrames

In [2]:
sucursales = pd.read_json('../data/data/sucursales.json', lines=True)
precios = pd.read_json('../data/data/precios.json', lines=True)
productos = pd.read_json('../data/data/productos.json', lines=True)

In [335]:
sucursales[sucursales['sucursalId']==116]

,_id,banderaDescripcion,banderaId,comercioId,comercioRazonSocial,direccion,id,lat,lng,localidad,provincia,sucursalId,sucursalNombre,sucursalTipo
483,5cbc698b7af152186c0cd322,COTO CICSA,1,12,Coto Centro Integral de ComercializaciÃ³n S.A.,Yerbal 1062,12-1-116,-34.621160,-58.445992,Caballito,AR-C,116,YERBAL,Supermercado
551,5cbc698b7af152186c0cd366,Market,2,10,INC S.A.,Av. Santa Fe 1954,10-2-116,-34.595813,-58.395425,Ciudad AutÃ³noma de Buenos Aires,AR-C,116,Av. Santa Fe 1954,Supermercado


In [175]:
sucursales_corregido = pd.read_csv('sucursales_corregido.csv')

## 2. Corregimos errores de loading de los datos

In [3]:
precios["fecha"]=list(map(lambda x: x["$date"],precios["fecha"]))
precios["_id"]=list(map(lambda x: x["$oid"],precios["_id"]))
productos["_id"]=list(map(lambda x: x["$oid"],productos["_id"]))
sucursales["_id"]=list(map(lambda x: x["$oid"],sucursales["_id"]))

## 3. Desglosamos la fecha en año, mes, dia y semana

In [4]:
precios["year"] = pd.DatetimeIndex(precios["fecha"]).year
precios["month"] = pd.DatetimeIndex(precios["fecha"]).month
precios["day"] = pd.DatetimeIndex(precios["fecha"]).day
precios["week"] = pd.DatetimeIndex(precios["fecha"]).week

## 4. Unificamos los datos en un DataFrame

In [5]:
precio_producto = precios.merge(productos,left_on='producto',right_on='id')

In [6]:
precio_producto_sucursal = precio_producto.merge(sucursales,left_on='sucursal',right_on='id')

In [7]:
precio_producto.columns.values

array(['_id_x', 'fecha', 'medicion', 'precio', 'producto', 'sucursal',
       'year', 'month', 'day', 'week', '_id_y', 'id', 'marca', 'nombre',
       'presentacion'], dtype=object)

In [8]:
precio_producto_sucursal.columns.values

array(['_id_x', 'fecha', 'medicion', 'precio', 'producto', 'sucursal',
       'year', 'month', 'day', 'week', '_id_y', 'id_x', 'marca', 'nombre',
       'presentacion', '_id', 'banderaDescripcion', 'banderaId',
       'comercioId', 'comercioRazonSocial', 'direccion', 'id_y', 'lat',
       'lng', 'localidad', 'provincia', 'sucursalId', 'sucursalNombre',
       'sucursalTipo'], dtype=object)

In [9]:
precio_producto_sucursal.head(3)

,_id_x,fecha,medicion,precio,producto,sucursal,year,month,day,week,...,comercioRazonSocial,direccion,id_y,lat,lng,localidad,provincia,sucursalId,sucursalNombre,sucursalTipo
0,5cba50f27af1523c35613042,2019-01-15T04:51:28.220Z,6,56.2,7790762052364,12-1-44,2019,1,15,3,...,Coto Centro Integral de ComercializaciÃ³n S.A.,Av. Monroe 3284,12-1-44,-34.563583,-58.468412,Belgrano,AR-C,44,MONROE,Supermercado
1,5cba5a537af1523c35976080,2019-02-23T09:53:40.735Z,9,56.2,7790762052364,12-1-44,2019,2,23,8,...,Coto Centro Integral de ComercializaciÃ³n S.A.,Av. Monroe 3284,12-1-44,-34.563583,-58.468412,Belgrano,AR-C,44,MONROE,Supermercado
2,5cba60367af1523c35ba838c,2018-11-08T16:35:49.030Z,1,56.2,7790762052364,12-1-44,2018,11,8,45,...,Coto Centro Integral de ComercializaciÃ³n S.A.,Av. Monroe 3284,12-1-44,-34.563583,-58.468412,Belgrano,AR-C,44,MONROE,Supermercado


In [10]:
len(precio_producto_sucursal['producto'].unique())

1000

## 5. Analizamos la cantidad de datos nulos por columna

In [11]:
# Analicemos la cantidad de nulos

def count_missing_data(dataframe):

    series_nulls = dataframe.isnull().sum()

    df_nulls = pd.DataFrame({'column': series_nulls.index, 'number_nulls': series_nulls.values})

    def number_rows_parent_dataframe(dataframe):
        return len(dataframe.index)

    def missing_data_porcentage(value, dataframe=dataframe):
        return value/number_rows_parent_dataframe(dataframe)*100

    df_nulls['percentage_nulls'] = df_nulls['number_nulls'].apply(missing_data_porcentage)

    return df_nulls

print(count_missing_data(precio_producto_sucursal))

                 column  number_nulls  percentage_nulls
0                 _id_x             0               0.0
1                 fecha             0               0.0
2              medicion             0               0.0
3                precio             0               0.0
4              producto             0               0.0
5              sucursal             0               0.0
6                  year             0               0.0
7                 month             0               0.0
8                   day             0               0.0
9                  week             0               0.0
10                _id_y             0               0.0
11                 id_x             0               0.0
12                marca             0               0.0
13               nombre             0               0.0
14         presentacion             0               0.0
15                  _id             0               0.0
16   banderaDescripcion             0           

In [12]:
precio_producto_sucursal[['producto','marca', 'nombre', 'presentacion']].head()

,producto,marca,nombre,presentacion
0,7790762052364,SANTA ANA,Vino Rosado Seleccion Especial Santa Ana 700 Ml,700.0 ml
1,7790762052364,SANTA ANA,Vino Rosado Seleccion Especial Santa Ana 700 Ml,700.0 ml
2,7790762052364,SANTA ANA,Vino Rosado Seleccion Especial Santa Ana 700 Ml,700.0 ml
3,7790762052364,SANTA ANA,Vino Rosado Seleccion Especial Santa Ana 700 Ml,700.0 ml
4,7790762052364,SANTA ANA,Vino Rosado Seleccion Especial Santa Ana 700 Ml,700.0 ml


### La siguiente investigacion consistira, en terminos generales, en analizar el comportamiento de las diferentes marcas de supermercados respecto a diferentes variables a lo largo de las diez mediciones en que consiste el dataset.

## 6. En que dias se relevaron que cantidad de sucursales por marca?

#### Creamos una nueva columna con solo el año, mes y dia de la fecha

In [83]:
precio_producto_sucursal['fecha_dt'] = pd.to_datetime(precio_producto_sucursal['fecha'])

precio_producto_sucursal['fecha_dt'] = precio_producto_sucursal['fecha_dt'].dt.date

precio_producto_sucursal['fecha_dt'] = pd.to_datetime(precio_producto_sucursal['fecha_dt'])

In [84]:
precio_producto_sucursal['fecha_dt'].dtype

dtype('<M8[ns]')

In [82]:
pd.to_datetime(precio_producto_sucursal['fecha_dt'].head(3))

0   2019-01-15
1   2019-02-23
2   2018-11-08
Name: fecha_dt, dtype: datetime64[ns]

In [65]:
# Hay fechas que me esta acomodando me esta intercambiando la posicion de los dias con los meses
precio_producto_sucursal[['sucursalId', 'fecha', 'year', 'month', 'day', 'fecha_dt']].head(3)

,sucursalId,fecha,year,month,day,fecha_dt
0,44,2019-01-15T04:51:28.220Z,2019,1,15,2019-01-15
1,44,2019-02-23T09:53:40.735Z,2019,2,23,2019-02-23
2,44,2018-11-08T16:35:49.030Z,2018,11,8,2018-11-08


#### Armamos un dataframe con la cantidad de sucursales que se relevaron por dia por marca

In [158]:
d = {'sucursalId': ['nunique']}

res = precio_producto_sucursal.groupby(['banderaDescripcion','medicion', 'fecha_dt']).agg(d)

res.columns = ['_'.join(col) for col in res.columns.values]

res = res.reset_index()

res.head()

,banderaDescripcion,medicion,fecha_dt,sucursalId_nunique
0,COTO CICSA,1,2018-11-05,15
1,COTO CICSA,1,2018-11-06,23
2,COTO CICSA,1,2018-11-07,17
3,COTO CICSA,1,2018-11-08,4
4,COTO CICSA,1,2018-11-09,3


In [115]:
res['fecha_dt'] = pd.to_datetime(res['fecha_dt'])

In [ ]:
def get_combination_closer_days(day):
    
    array_two_days_before = []
    
    array_one_before_one_after = []
    
    array_two_days_after = []
    
    one_day_before = day - pd.Timedelta(days=1)
    
    two_days_before = day - pd.Timedelta(days=2)
    
    one_day_after = day + pd.Timedelta(days=1)
    
    two_days_after = day + pd.Timedelta(days=2)
    
    array_two_days_before.append(one_day_before.values[0])
    array_two_days_before.append(two_days_before.values[0])
    
    array_one_before_one_after.append(one_day_before.values[0])
    array_one_before_one_after.append(one_day_after.values[0])
    
    array_two_days_after.append(one_day_after.values[0])
    array_two_days_after.append(two_days_after.values[0])
    
    return array_two_days_before, array_one_before_one_after, array_two_days_after

In [155]:
def get_days_with_max_sucursales(dataframe):
    
    array_brands = dataframe['banderaDescripcion'].unique()
    
    array_mediciones = dataframe['medicion'].unique()
    
    res_max = dataframe.loc[dataframe.groupby(['banderaDescripcion', 'medicion'])['sucursalId_nunique'].idxmax()]
    
    # Para cada cadena
    # y para cada medicion
    
    d = {'banderaDescripcion': array_brands}
    
    df = pd.DataFrame(data=d)
    
    for medicion in array_mediciones:
        
        
        array_max_cant_sucursales_relevadas_combination_three_days = []
        
        array_cant_sucursales_relevadas = []
        
        
        for supermarket in array_brands:
            
            # Obtenemos el dia con la mayor cantidad de sucursales relavadas
            
            try:
                day_max_sucursales = res_max[(res_max['banderaDescripcion']==supermarket) & (res_max['medicion']==medicion)]['fecha_dt']
            except IndexError:
                day_max_sucursales = 0
            
            array_max_sucursales = []
            
            # Si esta definido, day_max_sucursales es una pandas serie. No se la puede comparar con 0
            
            if not isinstance(day_max_sucursales, pd.Series) or day_max_sucursales.empty:
                
                array_max_sucursales.append(0)
                
                array_max_cant_sucursales_relevadas_combination_three_days.append(0)

                array_cant_sucursales_relevadas.append(0)
                
            else:    
            
                posible_date_combinations = get_combination_closer_days(day_max_sucursales)

                for combination in posible_date_combinations:

                    try:
                        day_max = dataframe[(dataframe['banderaDescripcion']==supermarket) & (dataframe['medicion']==medicion) & (dataframe['fecha_dt']==day_max_sucursales.values[0])]['sucursalId_nunique'].values[0]
                    except IndexError:
                        day_max = 0

                    try:
                        day_1 = dataframe[(dataframe['banderaDescripcion']==supermarket) & (dataframe['medicion']==medicion) & (dataframe['fecha_dt']==combination[0])]['sucursalId_nunique'].values[0]
                    except IndexError:
                        day_1 = 0

                    try:
                        day_2 = dataframe[(dataframe['banderaDescripcion']==supermarket) & (dataframe['medicion']==medicion) & (dataframe['fecha_dt']==combination[1])]['sucursalId_nunique'].values[0]
                    except IndexError:
                        day_2 = 0

                    array_max_sucursales.append(day_max + day_1 + day_2)
                
                
                array_max_cant_sucursales_relevadas_combination_three_days.append(max(array_max_sucursales))


                sucursales_relevadas_per_day = dataframe[(dataframe['banderaDescripcion']==supermarket) & (dataframe['medicion']==medicion)]['sucursalId_nunique'].values


                array_cant_sucursales_relevadas.append(sum(sucursales_relevadas_per_day))
        
        
        #Definimos los nombres de las columnas
        
        max_sucursales = str(medicion) + '_max_sucursales'
        
        total_sucursales = str(medicion) + '_total_sucursales'
        
        percentage = str(medicion) + '_percentage'
        
        df[max_sucursales] = array_max_cant_sucursales_relevadas_combination_three_days
        
        df[total_sucursales] = array_cant_sucursales_relevadas
        
        df[percentage] = df[max_sucursales] / df[total_sucursales]
            
    
    return df
      

In [156]:
# El dataframe que le pasamos como argumento consiste en la cantidad de sucursales que se relevaron por dia por marca,
# durante las diferentes mediciones

get_days_with_max_sucursales(res)

,banderaDescripcion,1_max_sucursales,1_total_sucursales,1_percentage,2_max_sucursales,2_total_sucursales,2_percentage,3_max_sucursales,3_total_sucursales,3_percentage,...,7_percentage,8_max_sucursales,8_total_sucursales,8_percentage,9_max_sucursales,9_total_sucursales,9_percentage,10_max_sucursales,10_total_sucursales,10_percentage
0,COTO CICSA,55,65,0.846154,44,57,0.771930,37,65,0.569231,...,0.784615,41,65,0.630769,37,65,0.569231,44,65,0.676923
1,Disco,21,26,0.807692,20,26,0.769231,15,26,0.576923,...,0.923077,22,26,0.846154,21,26,0.807692,21,26,0.807692
2,Express,9,9,1.000000,9,9,1.000000,7,9,0.777778,...,1.000000,7,9,0.777778,8,9,0.888889,9,9,1.000000
3,Hipermercado Carrefour,5,9,0.555556,3,9,0.333333,3,9,0.333333,...,0.666667,4,9,0.444444,4,9,0.444444,7,9,0.777778
4,JOSIMAR SUPERMERCADOS,1,1,1.000000,1,1,1.000000,1,1,1.000000,...,1.000000,1,1,1.000000,1,1,1.000000,1,1,1.000000
5,Jumbo,8,10,0.800000,7,10,0.700000,4,10,0.400000,...,1.000000,10,10,1.000000,6,10,0.600000,9,10,0.900000
6,Market,24,28,0.857143,19,28,0.678571,16,28,0.571429,...,0.857143,23,28,0.821429,22,28,0.785714,24,28,0.857143
7,Mi Changomas,1,1,1.000000,1,1,1.000000,1,1,1.000000,...,1.000000,1,1,1.000000,1,1,1.000000,0,0,NaN
8,Supermercados DIA,12,13,0.923077,13,13,1.000000,12,13,0.923077,...,1.000000,13,13,1.000000,10,13,0.769231,10,13,0.769231
9,Vea,11,12,0.916667,8,12,0.666667,11,12,0.916667,...,1.000000,12,12,1.000000,7,12,0.583333,11,12,0.916667


#### Se observa que la mayoria de las sucursales de cada marca son relevadas en un periodo de menos de 3 dias. Esto haria pensar que es dificil que las variaciones de precios observadas dentro de cada cadena se deban a la diferencia de dias que hubo entre que se relevo cada sucursal

## 7. Que cantidad de productos venden las diferentes marcas de supermercados?

In [13]:
precio_producto_sucursal['banderaDescripcion'].unique()

array(['COTO CICSA', 'Hipermercado Carrefour', 'Disco', 'Market',
       'Express', 'Jumbo', 'Vea', 'JOSIMAR SUPERMERCADOS',
       'Supermercados DIA', 'Walmart SuperCenter', 'Mi Changomas'],
      dtype=object)

In [14]:
def count_different_products(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        array_amount_diff_products = []
        
        for market in supermarkets:
            
            array_amount_diff_products.append(len(precio_producto_sucursal[(precio_producto_sucursal['medicion']==time) & (precio_producto_sucursal['banderaDescripcion']==market)]['producto'].unique()))
        
        df[time] = array_amount_diff_products
        
    return df        

In [15]:
count_different_products(precio_producto_sucursal)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,993,995,996,996,996,996,997,994,994,993
1,Hipermercado Carrefour,996,997,998,998,998,998,998,998,998,998
2,Disco,996,995,996,996,996,996,996,996,996,996
3,Market,997,996,997,997,997,997,997,997,997,997
4,Express,887,887,887,891,891,893,886,884,884,885
5,Jumbo,996,996,996,996,996,994,996,996,996,996
6,Vea,995,995,996,996,996,996,996,996,996,996
7,JOSIMAR SUPERMERCADOS,806,827,813,0,833,0,827,821,823,832
8,Supermercados DIA,760,760,760,764,763,770,768,810,810,805
9,Walmart SuperCenter,0,892,897,0,902,0,895,797,897,899


## 8. Cuantas sucursales se relevaron por marca? 

In [16]:
def number_sucursales_by_mkt_brand(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        d = {'sucursalId': ['nunique']}

        df_med = dataframe[dataframe['medicion']==time]

        res = df_med.groupby(['banderaDescripcion']).agg(d)

        res.columns = ['_'.join(col) for col in res.columns.values]

        res = res.reset_index()
        
        df = pd.merge(df, res, on='banderaDescripcion', how='left')
        
        df = df.rename(columns={'sucursalId_nunique': time})
        
    return df

In [17]:
number_sucursales_by_mkt_brand(precio_producto_sucursal)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,65.0,57,65,65.0,65,65.0,65,65,65,65.0
1,Hipermercado Carrefour,9.0,9,9,9.0,9,9.0,9,9,9,9.0
2,Disco,26.0,26,26,26.0,26,13.0,26,26,26,26.0
3,Market,28.0,28,28,28.0,28,28.0,28,28,28,28.0
4,Express,9.0,9,9,9.0,9,9.0,9,9,9,9.0
5,Jumbo,10.0,10,10,10.0,10,5.0,10,10,10,10.0
6,Vea,12.0,12,12,12.0,12,9.0,12,12,12,12.0
7,JOSIMAR SUPERMERCADOS,1.0,1,1,NaN,1,NaN,1,1,1,1.0
8,Supermercados DIA,13.0,13,13,12.0,13,13.0,13,13,13,13.0
9,Walmart SuperCenter,NaN,1,1,NaN,1,NaN,1,1,1,1.0


## 9. Hay sucursales dentro de una misma cadena que vendan menos/mas productos que otras?

In [160]:
# Tengo que contar la cantidad de productos por sucursal

d = {'producto': ['count']}
        
df_med = precio_producto_sucursal[precio_producto_sucursal['medicion']==1]

res_p = df_med.groupby(['banderaDescripcion', 'sucursalId']).agg(d)

res_p.columns = ['_'.join(col) for col in res_p.columns.values]

res_p = res_p.reset_index()

res_p.head()

,banderaDescripcion,sucursalId,producto_count
0,COTO CICSA,2,984
1,COTO CICSA,6,819
2,COTO CICSA,7,951
3,COTO CICSA,18,722
4,COTO CICSA,19,984


In [166]:
p = {'producto_count': ['min', 'max', 'mean']}

res_p_diff = res_p.groupby(['banderaDescripcion']).agg(p)

res_p_diff.columns = ['_'.join(col) for col in res_p_diff.columns.values]

res_p_diff = res_p_diff.reset_index()

res_p_diff

,banderaDescripcion,producto_count_min,producto_count_max,producto_count_mean
0,COTO CICSA,704,991,953.969231
1,Disco,916,975,952.461538
2,Express,774,852,826.666667
3,Hipermercado Carrefour,955,981,969.333333
4,JOSIMAR SUPERMERCADOS,806,806,806.000000
5,Jumbo,448,983,906.700000
6,Market,901,986,964.892857
7,Mi Changomas,636,636,636.000000
8,Supermercados DIA,759,759,759.000000
9,Vea,794,974,908.083333


In [163]:
res_p_diff['producto_count_min'].values

array([704, 916, 774, 955, 806, 448, 901, 636, 759, 794], dtype=int64)

In [169]:
def number_products_per_sucursal(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        array_prod_count_min = []
        
        array_prod_count_max = []
        
        array_prod_count_min = []
        
        d = {'producto': ['count']}
        
        df_med = dataframe[dataframe['medicion']==time]

        res_p = df_med.groupby(['banderaDescripcion', 'sucursalId']).agg(d)

        res_p.columns = ['_'.join(col) for col in res_p.columns.values]

        res_p = res_p.reset_index()
        
        # Agrupamos los datos por cadena
        
        p = {'producto_count': ['min', 'max', 'mean']}

        res_p_diff = res_p.groupby(['banderaDescripcion']).agg(p)

        res_p_diff.columns = ['_'.join(col) for col in res_p_diff.columns.values]

        res_p_diff = res_p_diff.reset_index()
        
        col_1 = str(time) + 'prod_count_min'
        
        col_2 = str(time) + 'prod_count_max'
        
        col_3 = str(time) + 'prod_count_mean'
        
        df = pd.merge(df, res_p_diff, on='banderaDescripcion', how='left')
        
        df = df.rename(columns={'producto_count_min': col_1, 'producto_count_max': col_2, 'producto_count_mean': col_3})
        
    return df
    

In [170]:
number_products_per_sucursal(precio_producto_sucursal)

,banderaDescripcion,1prod_count_min,1prod_count_max,1prod_count_mean,2prod_count_min,2prod_count_max,2prod_count_mean,3prod_count_min,3prod_count_max,3prod_count_mean,...,7prod_count_mean,8prod_count_min,8prod_count_max,8prod_count_mean,9prod_count_min,9prod_count_max,9prod_count_mean,10prod_count_min,10prod_count_max,10prod_count_mean
0,COTO CICSA,704.0,991.0,953.969231,707,993,953.087719,694,996,959.184615,...,954.153846,716,991,953.430769,708,993,954.107692,709.0,986.0,949.800000
1,Hipermercado Carrefour,955.0,981.0,969.333333,954,981,971.000000,963,985,975.888889,...,976.111111,964,978,972.777778,952,974,963.888889,952.0,974.0,965.333333
2,Disco,916.0,975.0,952.461538,925,970,952.153846,875,976,946.192308,...,942.038462,920,968,950.576923,938,975,957.192308,912.0,977.0,949.846154
3,Market,901.0,986.0,964.892857,939,986,965.392857,951,986,971.178571,...,958.607143,895,973,948.892857,862,973,937.607143,863.0,962.0,941.035714
4,Express,774.0,852.0,826.666667,780,851,825.888889,759,858,826.333333,...,823.333333,730,845,811.000000,702,837,803.111111,715.0,841.0,804.777778
5,Jumbo,448.0,983.0,906.700000,453,977,900.400000,446,976,901.800000,...,890.300000,461,977,902.700000,479,979,904.200000,481.0,978.0,904.700000
6,Vea,794.0,974.0,908.083333,799,960,903.583333,792,981,907.750000,...,901.416667,777,961,902.333333,801,969,909.083333,794.0,974.0,906.333333
7,JOSIMAR SUPERMERCADOS,806.0,806.0,806.000000,827,827,827.000000,813,813,813.000000,...,827.000000,821,821,821.000000,823,823,823.000000,832.0,832.0,832.000000
8,Supermercados DIA,759.0,759.0,759.000000,755,760,759.615385,759,760,759.230769,...,768.000000,809,810,809.076923,808,810,809.461538,803.0,805.0,804.153846
9,Walmart SuperCenter,NaN,NaN,NaN,892,892,892.000000,897,897,897.000000,...,895.000000,797,797,797.000000,897,897,897.000000,899.0,899.0,899.000000


## 9.1 Donde estan las sucursales con menos/mas productos? 
(vamos a definir las sucursales con menos productos como las que tienen menos productos que la media)

In [180]:
sucursales_corregido.head(2)

,Unnamed: 0,_id,banderaDescripcion,banderaId,comercioId,comercioRazonSocial,direccion,id,lat,lng,localidad,provincia,sucursalId,sucursalNombre,sucursalTipo,geometry,localidad_lower
0,0,5cbc698b7af152186c0cd13f,Supermercados DIA,1,15,DIA Argentina S.A,Av Dr. Ricardo Balbin 4881,15-1-480,-34.552118,-58.498415,saavedra,AR-C,480,480 - Saavedra,Autoservicio,POINT (-58.4984145 -34.5521177),saavedra
1,1,5cbc698b7af152186c0cd140,DEHEZA S.A.I.C.F. e I.,1,3,Deheza S.A.I.C.F. e I.,San Juan Bautista De La Salle 4356,3-1-1506,-34.559454,-58.505028,saavedra,AR-C,1506,GRAL PAZ - NORTE,Autoservicio,POINT (-58.505028 -34.559454),saavedra


In [178]:
# Hay nombre de localidades que estan con y sin mayusculas. Pasamos todo a minusculas

sucursales_corregido['localidad_lower'] = sucursales_corregido['localidad'].str.lower()

In [183]:
sucursales_corregido[['sucursalId', 'localidad_lower']].head()

,sucursalId,localidad_lower
0,480,saavedra
1,1506,saavedra
2,675,nuñez
3,1507,saavedra
4,29,saavedra


In [289]:
d = {'producto': ['count']}
        
df_med = precio_producto_sucursal[precio_producto_sucursal['medicion']==1]

res_p = df_med.groupby(['banderaDescripcion', 'sucursalId']).agg(d)

res_p.columns = ['_'.join(col) for col in res_p.columns.values]

res_p = res_p.reset_index()

#res_p = pd.merge(res_p, sucursales_corregido[['sucursalId', 'localidad_lower']], on='sucursalId', how='left')

res_p.head(10)

,banderaDescripcion,sucursalId,producto_count
0,COTO CICSA,2,984
1,COTO CICSA,6,819
2,COTO CICSA,7,951
3,COTO CICSA,18,722
4,COTO CICSA,19,984
5,COTO CICSA,20,715
6,COTO CICSA,22,973
7,COTO CICSA,24,982
8,COTO CICSA,25,979
9,COTO CICSA,26,839


In [291]:
res_p[(res_p['banderaDescripcion']=='COTO CICSA') & (res_p['producto_count']<755)]

,banderaDescripcion,sucursalId,producto_count
3,COTO CICSA,18,722
5,COTO CICSA,20,715
44,COTO CICSA,116,704


In [202]:
def q1(x):
    return x.quantile(0.25)

p = {'producto_count': [ 'min']}

res_p_diff = res_p.groupby(['banderaDescripcion']).agg(p)

res_p_diff.columns = ['_'.join(col) for col in res_p_diff.columns.values]

res_p_diff = res_p_diff.reset_index()

res_p_diff

,banderaDescripcion,producto_count_min
0,COTO CICSA,704
1,Disco,916
2,Express,774
3,Hipermercado Carrefour,955
4,JOSIMAR SUPERMERCADOS,806
5,Jumbo,448
6,Market,901
7,Mi Changomas,636
8,Supermercados DIA,759
9,Vea,794


In [279]:
res_p_diff[res_p_diff['banderaDescripcion']=='COTO CICSA']['producto_count_min'].values[0]

704

In [204]:
res_p_diff[res_p_diff['banderaDescripcion']=='COTO CICSA']['producto_count_min'].values[0]

704

In [201]:
p = {'localidad_lower': ['count']}

res_p_local = res_p[(res_p['producto_count']<705) & (res_p['banderaDescripcion']=='COTO CICSA')].groupby(['banderaDescripcion', 'localidad_lower']).agg(p)

res_p_local.columns = ['_'.join(col) for col in res_p_local.columns.values]

res_p_local = res_p_local.reset_index()

res_p_local

,banderaDescripcion,localidad_lower,localidad_lower_count
0,COTO CICSA,caballito,1
1,COTO CICSA,recoleta,1


In [208]:
d = {'col1': ['a', 'b']}
df = pd.DataFrame(data=d)

d_1 = {'col1': ['a', 'b'], 'col2': [1, 2]}
df_1 = pd.DataFrame(data=d_1)

d_3 = {'col1': ['b', 'b'], 'col3': [ 2, 3]}

df_3 = pd.DataFrame(data=d_3)

df = pd.merge(df, df_1, on='col1', how='left')

df = pd.merge(df, df_3, on='col1', how='left')

df

,col1,col2,col3
0,a,1,NaN
1,b,2,2.0
2,b,2,3.0


In [224]:
d = {'col1': ['a', 'b']}
df = pd.DataFrame(data=d)

d_1 = {'col2': ['12', '2']}
df_1 = pd.DataFrame(data=d_1)

df = pd.merge(df, df_1, right_on='col2', left_on='col1', how='left')
df

,col1,col2
0,a,NaN
1,b,NaN


In [210]:
d = {'banderaDescripcion': [], 'locations': []}
    
df = pd.DataFrame(data=d)

d_1 = {'banderaDescripcion': [1], 'locations': [2]}

df_1 = pd.DataFrame(data=d_1)

df = pd.concat([df, df_1], ignore_index=True)

df

,banderaDescripcion,locations
0,1.0,2.0


In [ ]:
# Obtengo el mean

# Filtro el dataframe por el mean

# Cuento cuantas sucursales tiene cada marca por barrio

# Tengo que concatenar esos sub-dataframes

# y luego hacer el merge

# En ultima instancia tenemos que contar la cantidad de sucursales con menos productos que tiene cada localidad

In [249]:
def build_dataframe_brands_locations(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    array_supermarkets = []
    
    localidades = dataframe['localidad_lower'].unique().tolist()
    
    array_locations = []
    
    for market in supermarkets:
        
        array_supermarkets = array_supermarkets + [market]*len(localidades)
        
        array_locations = array_locations + localidades
    
    d = {'banderaDescripcion': array_supermarkets, 'localidades': array_locations}
    
    df = pd.DataFrame(data=d)
    
    return df

In [254]:
df_banderas_localidades = build_dataframe_brands_locations(sucursales_corregido)

df_banderas_localidades.head()

,banderaDescripcion,localidades
0,Supermercados DIA,saavedra
1,Supermercados DIA,nuñez
2,Supermercados DIA,villa pueyrredon
3,Supermercados DIA,coghlan
4,Supermercados DIA,villa urquiza


In [310]:
def get_locality_sucursal_with_min_amount_products(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        # Calculamos la cantidad de productos por sucursal
        
        array_sucursales_min_amount_prod = []
        
        d = {'producto': ['count']}
        
        df_med = dataframe[dataframe['medicion']==time]

        res_p = df_med.groupby(['banderaDescripcion', 'sucursalId']).agg(d)

        res_p.columns = ['_'.join(col) for col in res_p.columns.values]

        res_p = res_p.reset_index() 
        
        p = {'producto_count': [ 'min']}

        res_p_min = res_p.groupby(['banderaDescripcion']).agg(p)

        res_p_min.columns = ['_'.join(col) for col in res_p_min.columns.values]

        res_p_min = res_p_min.reset_index()
        
        for market in supermarkets:
            
            try:
                
                min_amount = (res_p_min[res_p_min['banderaDescripcion']==market]['producto_count_min'].values[0] + 1)
                
                res_min = res_p[(res_p['producto_count']<min_amount) & (res_p['banderaDescripcion']==market)]['sucursalId']
                
                array_sucursales_min_amount_prod.append(res_min.values.tolist())
                
            except IndexError:
            
                array_sucursales_min_amount_prod.append([0])
            
        
        df[time] = array_sucursales_min_amount_prod

    return df

In [311]:
get_locality_sucursal_with_min_amount_products(precio_producto_sucursal)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,[116],[18],[116],[116],[116],[20],[116],[116],[116],[116]
1,Hipermercado Carrefour,[271],[30],[214],[26],[271],[220],[220],[5],[3],[271]
2,Disco,[237],[237],[630],[237],[237],[630],[630],[237],[48],[48]
3,Market,[118],[237],[189],[118],[258],[258],[519],[150],[386],[390]
4,Express,[250],[250],[250],[250],[250],[250],[250],[250],[250],[250]
5,Jumbo,[5276],[5276],[5276],[5276],[5276],[5961],[5276],[5276],[5276],[5276]
6,Vea,[19],[19],[19],[19],[19],[24],[19],[19],[19],[19]
7,JOSIMAR SUPERMERCADOS,[5],[5],[5],[0],[5],[0],[5],[5],[5],[5]
8,Supermercados DIA,"[1066, 1072, 1074, 1076, 1080, 1086, 1091, 109...",[1066],"[1072, 1074, 1076, 1086, 1091, 1093, 1531, 153...","[1066, 1072, 1074, 1076, 1080, 1086, 1091, 109...","[1091, 1532]","[1072, 1074, 1080, 1093]","[1066, 1072, 1074, 1076, 1080, 1086, 1091, 109...","[1066, 1072, 1074, 1076, 1080, 1086, 1091, 109...","[1076, 1538]",[1532]
9,Walmart SuperCenter,[0],[2997],[2997],[0],[2997],[0],[2997],[2997],[2997],[2997]


In [320]:
def get_sucursales_bandera(dataframe, market):
    
    array_sucursales = []
    
    time_frames = sorted(precio_producto_sucursal['medicion'].unique())
    
    for time in time_frames:

        array_sucursales = array_sucursales + dataframe[dataframe['banderaDescripcion']==market].loc[:,time].values[0]
            
    counter=collections.Counter(array_sucursales)

    d = {'sucursales': list(counter.keys()), 'freq': list(counter.values())}

    df = pd.DataFrame(data=d)
    
    array_bandera = [market]*len(df['sucursales'].values.tolist())
    
    df['banderaDescripcion'] = array_bandera
    
    return df

In [325]:
get_sucursales_bandera(get_locality_sucursal_with_min_amount_products(precio_producto_sucursal), 'COTO CICSA')

,sucursales,freq,banderaDescripcion
0,116,8,COTO CICSA
1,18,1,COTO CICSA
2,20,1,COTO CICSA


In [330]:
def get_sucursales_by_bandera(dataframe):
    
    d = {'sucursales': [], 'freq': [], 'banderaDescripcion': []}
    
    df = pd.DataFrame(data=d)
    
    supermarkets = precio_producto_sucursal['banderaDescripcion'].unique()
    
    for market in supermarkets:
        
        df_suc = get_sucursales_bandera(get_locality_sucursal_with_min_amount_products(dataframe), market)
        
        df = pd.concat([df, df_suc], ignore_index=True)
        
    return df    

In [337]:
df_sucursales_min_prod = get_sucursales_by_bandera(precio_producto_sucursal)

df_sucursales_min_prod.head()

,sucursales,freq,banderaDescripcion
0,116.0,8.0,COTO CICSA
1,18.0,1.0,COTO CICSA
2,20.0,1.0,COTO CICSA
3,271.0,3.0,Hipermercado Carrefour
4,30.0,1.0,Hipermercado Carrefour


In [347]:
df_sucursales_min_prod['sucId_bandera'] = df_sucursales_min_prod['sucursales'].astype(int).astype(str) + '_' + df_sucursales_min_prod['banderaDescripcion']

df_sucursales_min_prod['sucId_bandera'].head()

0                116_COTO CICSA
1                 18_COTO CICSA
2                 20_COTO CICSA
3    271_Hipermercado Carrefour
4     30_Hipermercado Carrefour
Name: sucId_bandera, dtype: object

In [342]:
sucursales_corregido['sucId_bandera'] = sucursales_corregido['sucursalId'].astype(str) + '_' + sucursales_corregido['banderaDescripcion']

In [348]:
df_sucursales_min_prod = pd.merge(df_sucursales_min_prod, sucursales_corregido[['sucId_bandera', 'localidad_lower']], on='sucId_bandera', how='left')

df_sucursales_min_prod.head()

,sucursales,freq,banderaDescripcion,sucId_bandera,localidad,localidad_lower_x,localidad_lower_y
0,116.0,8.0,COTO CICSA,116_COTO CICSA,NaN,NaN,caballito
1,18.0,1.0,COTO CICSA,18_COTO CICSA,NaN,NaN,once
2,20.0,1.0,COTO CICSA,20_COTO CICSA,NaN,NaN,belgrano
3,271.0,3.0,Hipermercado Carrefour,271_Hipermercado Carrefour,NaN,NaN,belgrano
4,30.0,1.0,Hipermercado Carrefour,30_Hipermercado Carrefour,NaN,NaN,mataderos


In [352]:
def get_frecuency_localidades(dataframe):
    
    p = {'localidad_lower_y': [ 'count']}

    res_p_count = dataframe.groupby(['localidad_lower_y']).agg(p)

    res_p_count.columns = ['_'.join(col) for col in res_p_count.columns.values]

    res_p_count = res_p_count.reset_index()
    
    return res_p_count

In [353]:
get_frecuency_localidades(df_sucursales_min_prod)

,localidad_lower_y,localidad_lower_y_count
0,almagro,2
1,balvanera,2
2,barracas,2
3,belgrano,2
4,caballito,3
5,colegiales,1
6,mataderos,4
7,monserrat,1
8,once,1
9,palermo,4


### Pareciera que las sucursales con menor cantidad de productos de cada cadena no estan concentradas en los mismos barrios

## Vamos a tratar de inferir la existencia de estrategias competitivas de las diferentes marcas de supermercados:

  ###  - Adaptan los supermercados el precio de sus sucursales al lugar donde estan?
  ###  - Existe una cadena que este intentando diferenciarse de las demas a traves del precio?
  ###  - Existen cadenas que intenten diferenciarse de las demas mantentiendo el precio de ciertas categorias de productos mas baratos que el resto?

## Nota
### Los puntos 10,11 y 12 habria que volver a hacerlos solamente tomando el grupo de sucursales que viene dado por el punto 6. Ademas, hay que hacer el punto 10.2

### Lo que podemos hacer es sacar del dataset las filas que no corresponden a esas sucursales y volver a correr las funciones definidas para 10, 11 y 12

## 10. Hay diferencias de precios entre las diferentes sucursales de una misma marca?

In [19]:
def get_diff_prices_products_sucursales(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        array_amount_prod_with_diff = []
        
        array_mean_diff = []
        
        d = {'precio': ['min', 'max']}
        
        df_med = dataframe[dataframe['medicion']==time]

        res = df_med.groupby(['banderaDescripcion', 'producto']).agg(d)

        res.columns = ['_'.join(col) for col in res.columns.values]

        res = res.reset_index()

        res['diff'] = res['precio_max'] - res['precio_min']

        res['diff_per'] = (res['precio_max'] - res['precio_min'])/res['precio_min']*100
        
        for market in supermarkets:
            
            array_amount_prod_with_diff.append(len(res[(res['banderaDescripcion']==market) & (res['diff']>0)]['diff'].values))
            
            array_mean_diff.append(np.mean(res[(res['banderaDescripcion']==market) & (res['diff']>0)]['diff'].values))
        
        col_amount = str(time) + '_amount_prod_diff'
        
        col_mean = str(time) + '_amount_mean_diff'
        
        df[col_amount] = array_amount_prod_with_diff
        
        df[col_mean] = array_mean_diff
        
    return df        

In [20]:
get_diff_prices_products_sucursales(precio_producto_sucursal)

c:\users\bangho\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\bangho\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,banderaDescripcion,1_amount_prod_diff,1_amount_mean_diff,2_amount_prod_diff,2_amount_mean_diff,3_amount_prod_diff,3_amount_mean_diff,4_amount_prod_diff,4_amount_mean_diff,5_amount_prod_diff,...,6_amount_prod_diff,6_amount_mean_diff,7_amount_prod_diff,7_amount_mean_diff,8_amount_prod_diff,8_amount_mean_diff,9_amount_prod_diff,9_amount_mean_diff,10_amount_prod_diff,10_amount_mean_diff
0,COTO CICSA,33,7.144242,168,8.563571,110,10.023909,53,14.805849,36,...,318,9.108082,116,11.385690,34,13.705588,91,10.905385,84,17.801310
1,Hipermercado Carrefour,767,9.355189,764,8.914869,771,9.139468,692,6.505116,737,...,800,6.574100,648,6.964120,732,4.857268,842,6.544537,759,6.049736
2,Disco,394,7.384645,432,6.528056,396,6.613157,457,5.971707,343,...,374,7.583342,369,6.749377,371,6.236981,377,6.245119,477,6.757254
3,Market,655,10.093634,656,8.775152,626,8.328642,455,5.908923,562,...,556,6.376583,509,6.491415,447,5.812170,522,8.030498,389,5.744242
4,Express,132,4.710379,13,9.388462,165,6.343030,138,8.843478,162,...,57,8.828070,144,10.703542,189,7.819630,170,10.016588,115,9.069652
5,Jumbo,464,8.688642,470,7.525702,418,8.380144,413,7.697433,426,...,364,8.044203,395,9.697367,424,9.083396,439,10.844897,515,7.564194
6,Vea,159,5.663208,160,5.305312,152,5.699474,206,5.226553,162,...,201,5.198756,113,5.176991,186,6.966290,201,7.255075,202,4.845693
7,JOSIMAR SUPERMERCADOS,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
8,Supermercados DIA,50,6.058400,4,15.500000,110,6.653273,21,8.653810,29,...,64,7.362812,5,9.390000,5,15.760000,38,5.711316,43,5.400465
9,Walmart SuperCenter,0,NaN,0,NaN,0,NaN,0,NaN,0,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN


## 10.2 Que sucursales son las que tienen los menores precios? 

(estan asociadas a algun barrio en especial? Podemos analizar el barrio de la sucursal mas cara vs la sucursal mas barata. Ademas, podemos analizar el dia en que ambas sucursales fueron relevadas)

In [174]:
precio_producto_sucursal[['banderaDescripcion', 'sucursalId', 'producto', 'precio', 'fecha_dt']].head(10)

,banderaDescripcion,sucursalId,producto,precio,fecha_dt
0,COTO CICSA,44,7790762052364,56.2,2019-01-15
1,COTO CICSA,44,7790762052364,56.2,2019-02-23
2,COTO CICSA,44,7790762052364,56.2,2018-11-08
3,COTO CICSA,44,7790762052364,56.2,2019-02-09
4,COTO CICSA,44,7790762052364,56.2,2018-12-23
5,COTO CICSA,44,7790762052364,56.2,2019-03-01
6,COTO CICSA,44,7790762052364,56.2,2018-12-10
7,COTO CICSA,44,7790762052364,56.2,2018-11-16
8,COTO CICSA,44,7790762052364,56.2,2019-01-22
9,COTO CICSA,44,7790762052364,56.2,2018-12-02


## 11. Que cadena vende la mayor cantidad de productos al menor precio?

(Ya que los precios de los mismos varian entre sucursales de una misma cadena, vamos a tomar el precio minimo que una cadena venda un producto)

In [21]:
def get_amount_products_with_min_price_by_mkt_brand(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        array_amount_prod_with_min_price = []
        
        d_pr = {'precio': ['min']}

        df_med = dataframe[dataframe['medicion']==time]

        res = df_med.groupby(['banderaDescripcion', 'producto']).agg(d_pr)

        res.columns = ['_'.join(col) for col in res.columns.values]

        res = res.reset_index()
        
        res_min = res.loc[res.groupby(['producto'])['precio_min'].idxmin()]
        
        for market in supermarkets:
            
            array_amount_prod_with_min_price.append(len(res_min[res_min['banderaDescripcion']==market].index))
        
        df[time] = array_amount_prod_with_min_price
    
    return df
        

In [22]:
get_amount_products_with_min_price_by_mkt_brand(precio_producto_sucursal)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,55,68,62,120,61,127,78,58,62,74
1,Hipermercado Carrefour,269,296,328,493,318,375,318,259,316,331
2,Disco,24,28,14,18,22,18,22,17,17,25
3,Market,152,75,114,64,98,79,91,73,72,99
4,Express,91,109,99,124,80,109,102,79,79,67
5,Jumbo,11,7,10,10,11,19,16,16,33,9
6,Vea,37,39,17,18,24,52,19,46,29,40
7,JOSIMAR SUPERMERCADOS,161,168,181,0,205,0,169,208,198,225
8,Supermercados DIA,137,114,103,153,113,221,94,139,115,91
9,Walmart SuperCenter,0,40,25,0,27,0,42,52,44,39


## 12. Que tipo de productos son los que cada cadena vende a menor precio?

Mostramos los productos que siempre estuvieron mas baratos en cada cadena (tambien podemos analizar diferentes periodos: al menos en 7 mediciones, 6, etc)

In [23]:
def get_products_with_min_price_by_mkt_brand(dataframe):
    
    supermarkets = dataframe['banderaDescripcion'].unique()
    
    time_frames = sorted(dataframe['medicion'].unique())
    
    d = {'banderaDescripcion': supermarkets}
    
    df = pd.DataFrame(data=d)
    
    for time in time_frames:
        
        array_amount_prod_with_min_price = []
        
        d_pr = {'precio': ['min']}

        df_med = dataframe[dataframe['medicion']==time]

        res = df_med.groupby(['banderaDescripcion', 'producto']).agg(d_pr)

        res.columns = ['_'.join(col) for col in res.columns.values]

        res = res.reset_index()
        
        res_min = res.loc[res.groupby(['producto'])['precio_min'].idxmin()]
        
        for market in supermarkets:
            
            array_amount_prod_with_min_price.append(res_min[res_min['banderaDescripcion']==market]['producto'].values.tolist())
        
        df[time] = array_amount_prod_with_min_price
    
    return df

In [24]:
get_products_with_min_price_by_mkt_brand(precio_producto_sucursal)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,"[0000077940131, 12-1-2800000937881, 4005808352...","[0000077940131, 12-1-2800000937881, 4005808352...","[12-1-2800000937881, 4005808352630, 7622300398...","[0000077940131, 0000080052760, 12-1-2800000937...","[12-1-2800000937881, 4005808352630, 7613035379...","[0000080052760, 12-1-2800000937881, 4005808352...","[0000080052760, 12-1-2800000937881, 4005808352...","[12-1-2800000937881, 4005808352630, 7613034191...","[12-1-2800000937881, 4005808352630, 7613034191...","[12-1-2800000937881, 4005808352630, 7501059273..."
1,Hipermercado Carrefour,"[0000077903518, 0000078924468, 0000080052760, ...","[0000077903518, 0000078924468, 0000078924529, ...","[0000075032715, 0000077903518, 3014260273903, ...","[0000075027513, 0000077903518, 0000077940704, ...","[0000075027513, 0000077903518, 0041789001918, ...","[0000075027513, 0000077903518, 0000077940704, ...","[0000075032715, 0000077903518, 0000078924468, ...","[0000075027513, 0000078924468, 0000078924529, ...","[0000075027513, 0000078924468, 0000078924529, ...","[0000075027513, 0000078924468, 0000078924529, ..."
2,Disco,"[0038000846731, 7613035161580, 7613035379213, ...","[0038000846731, 7506339337532, 7622300840259, ...","[0038000846731, 7500435004664, 7622300840273, ...","[7500435112734, 7622300840259, 7790070410146, ...","[0000077940131, 7500435112741, 7622300840259, ...","[0000077940131, 0038000846731, 7790040994904, ...","[0000077940131, 0038000846731, 7622300833930, ...","[0038000846731, 7790040001022, 7790040102989, ...","[0038000846731, 7790040001022, 7790040102989, ...","[0038000846731, 7500435019248, 7622210649287, ..."
3,Market,"[0000077940704, 4005808979813, 7500435012034, ...","[0000075027513, 0000075032715, 7500435012041, ...","[0000075027513, 0000078924468, 0000078924529, ...","[0038000846731, 4005808979813, 7500435012034, ...","[0000080052760, 0038000846731, 4005808979813, ...","[4005808979813, 7500435012034, 7500435012041, ...","[0000075027513, 4005808315093, 7500435012034, ...","[0000080052760, 7500435106405, 7790040439108, ...","[0000075032715, 0000080052760, 7500435106405, ...","[0000075032715, 0000080052760, 0041789002922, ..."
4,Express,"[0000040084107, 0000075032715, 0041789001918, ...","[0000040084107, 0041789001918, 0070501051054, ...","[0000040084107, 0041789001918, 0070501051054, ...","[0000040084107, 0000078924468, 0000078924529, ...","[0000040084107, 0070501051054, 7500435019231, ...","[0000040084107, 0000077953063, 0000077953070, ...","[0000040084107, 7500435019231, 7501059273252, ...","[7500435019231, 7506309878997, 7613034449993, ...","[7500435112741, 7506309878997, 7613034413154, ...","[0000040084107, 7500435112741, 7613034416902, ..."
5,Jumbo,"[7500435019231, 7622210812797, 7622210813534, ...","[7500435112741, 7790040711105, 7790150006153, ...","[0000077940131, 7790010616775, 7790040711105, ...","[7500435112741, 7790520014184, 7790520014191, ...","[7500435004657, 7500435004664, 7622210812797, ...","[7500435012461, 7622210812797, 7790040931206, ...","[7500435012461, 7622210812797, 7790010596602, ...","[7500435012461, 7622210812797, 7622210813534, ...","[0000077940131, 3014260273903, 3014260273910, ...","[7613034191182, 7790230033031, 7790480008261, ..."
6,Vea,"[4005808980239, 7501065904522, 7506295302292, ...","[4005808980239, 7501065904522, 7506295302292, ...","[4005808980239, 7501065904522, 7506295302292, ...","[4005808980239, 7501065904522, 7506295302292, ...","[7501065904522, 7506295302292, 7622210813534, ...","[3014260273903, 3014260273910, 7500435019248, ...","[7622300742584, 7790040929807, 7790230033031, ...","[0000075032715, 3014260273903, 3014260273910, ...","[7500435023306, 7500435112734, 7500435112758, ...","[7613035379213, 7622300868512, 7622300868543, ..."
7,JOSIMAR SUPERMERCADOS,"[0099176316565, 4005808315093, 7500435004633, ...","[0000077940704, 0000077953063, 0000077953070, ...","[0000077940704, 0000077953063, 0000077953070, ...",[],"[0000077940704, 0000077953063, 0000077953070, ...",[],"[

In [25]:
def get_unique_prod_with_min_price_by_mkt_brand(dataframe, mkt_brand, number_periods):
    
    array_products = []
    
    time_frames = sorted(precio_producto_sucursal['medicion'].unique())
    
    for time in time_frames:
        
        array_products = array_products + dataframe[dataframe['banderaDescripcion']==mkt_brand].loc[:,time].values[0]
        
    counter=collections.Counter(array_products)
    
    d = {'products': list(counter.keys()), 'freq': list(counter.values())}
    
    df = pd.DataFrame(data=d)
    
    df_with_out_duplicates = precio_producto_sucursal[['producto', 'marca', 'nombre']].drop_duplicates(subset=['producto', 'marca', 'nombre'])
    
    df = pd.merge(df, df_with_out_duplicates, left_on='products', right_on='producto', how='left' )
    
    return df[df['freq']>= number_periods]

In [26]:
min_price_by_mkt = get_products_with_min_price_by_mkt_brand(precio_producto_sucursal)

min_price_by_mkt.head(2)

,banderaDescripcion,1,2,3,4,5,6,7,8,9,10
0,COTO CICSA,"[0000077940131, 12-1-2800000937881, 4005808352...","[0000077940131, 12-1-2800000937881, 4005808352...","[12-1-2800000937881, 4005808352630, 7622300398...","[0000077940131, 0000080052760, 12-1-2800000937...","[12-1-2800000937881, 4005808352630, 7613035379...","[0000080052760, 12-1-2800000937881, 4005808352...","[0000080052760, 12-1-2800000937881, 4005808352...","[12-1-2800000937881, 4005808352630, 7613034191...","[12-1-2800000937881, 4005808352630, 7613034191...","[12-1-2800000937881, 4005808352630, 7501059273..."
1,Hipermercado Carrefour,"[0000077903518, 0000078924468, 0000080052760, ...","[0000077903518, 0000078924468, 0000078924529, ...","[0000075032715, 0000077903518, 3014260273903, ...","[0000075027513, 0000077903518, 0000077940704, ...","[0000075027513, 0000077903518, 0041789001918, ...","[0000075027513, 0000077903518, 0000077940704, ...","[0000075032715, 0000077903518, 0000078924468, ...","[0000075027513, 0000078924468, 0000078924529, ...","[0000075027513, 0000078924468, 0000078924529, ...","[0000075027513, 0000078924468, 0000078924529, ..."


In [27]:
get_unique_prod_with_min_price_by_mkt_brand(min_price_by_mkt, 'Hipermercado Carrefour', 8)

,products,freq,producto,marca,nombre
3,7500435004657,8,7500435004657,ARIEL,Jabon Liquido Ropa Agua Fria Ariel 3 Lt
10,7500435023306,8,7500435023306,ALWAYS,Toalla Femenina Plus Seca con Alas Always 16 Un
19,7622210649249,10,7622210649249,TERRABUSI,Galletitas Surtidas Chocolate Terrabusi Varied...
20,7622210649287,8,7622210649287,TERRABUSI,Galletitas Clasicas Surtidas Terrabusi Varieda...
23,7622300847265,9,7622300847265,EXPRESS,Galletitas Clasicas Express Pack 5 Un 540 Gr
24,7622300847340,10,7622300847340,EXPRESS,Galletitas Light Express 110 Gr
25,7622300864934,10,7622300864934,OREO,Galletitas Chocolate con Relleno de Vainilla O...
32,7622300869915,10,7622300869915,OREO,Galletitas Chocolate con Relleno de Chocolate ...
34,7702018652372,10,7702018652372,ORAL B,Cepillo Dental Oral B Classic 1 Un
35,7790010570541,9,7790010570541,JOHNSON'S,Shampoo para Bebe Cabellos Claros Mollusk John...
